## Full List

In [2]:
import pandas as pd
import numpy as np
from datetime import datetime
import glob
import os

In [3]:
offline_dir = r"C:\Users\Kitty Wong\02 Analysis\01 Data\offlinetran"
online_dir = r"C:\Users\Kitty Wong\02 Analysis\01 Data\onlinetran"

offline_files = glob.glob(os.path.join(offline_dir, "Daily-Transaction-Summary-*.csv"))
online_files = glob.glob(os.path.join(online_dir, "Daily-Transaction-Summary-*.csv"))

trans_df = pd.read_csv(r"C:\Users\Kitty Wong\Downloads\trans.csv")
Excluded_df = pd.read_csv(r"C:\Users\Kitty Wong\02 Analysis\01 Data\Excluded.csv")
offline_daily_summary_df = pd.read_csv(offline_files[-1])
online_daily_summary_df = pd.read_csv(online_files[-1])
MCC_Classification_List_df = pd.read_excel(r"C:\Users\Kitty Wong\02 Analysis\01 Data\MCC Classification List.xlsx")
merchants_3000to_be_excluded = pd.read_excel(r"C:\Users\Kitty Wong\Downloads\3000 Merchants.xlsx")

### concat offline & online daily summary tables

In [297]:
online_daily_summary_df.rename(columns={'Legal Name': 'Shop Name'}, inplace=True)
daily_summary_df = pd.concat([offline_daily_summary_df, online_daily_summary_df])

### MCC Classification list

In [298]:
MCC_Classification_List_df = MCC_Classification_List_df[['MCC Code', 'DescriptionMCC description']]
MCC_Classification_List_df.rename(columns={'MCC Code': 'MCC', 'DescriptionMCC description': 'MCC Name'}, inplace=True)

daily_summary_filtered_df = daily_summary_df[['Create Date', 'Merchant ID', 'Shop Name', 'status', 'MCC']]
MCC_merged_df = daily_summary_filtered_df.merge(MCC_Classification_List_df, on='MCC', how='left')

### Create an Age column

In [299]:
MCC_merged_df['Create Date'] = pd.to_datetime(MCC_merged_df['Create Date'], format='%m/%d/%Y %H:%M:%S')
latest_date = datetime.now()

MCC_merged_df['Age'] = ((latest_date.year - MCC_merged_df['Create Date'].dt.year) * 12 + 
                        (latest_date.month - MCC_merged_df['Create Date'].dt.month))


### pivot trans.csv + merge

In [300]:
trans_pivot_df = trans_df.pivot(index='merchant_id', columns='transaction_month', values='net_amount').reset_index()
trans_pivot_df.rename(columns={'merchant_id': 'Merchant ID'}, inplace=True)

merged_df = MCC_merged_df.merge(trans_pivot_df, on='Merchant ID', how='left')
merged_df = merged_df.fillna(0)

### exclude merchants from Excluded.csv

In [301]:
excluded_merchants = Excluded_df['Merchant ID'].tolist()
merged_df = merged_df[~merged_df['Merchant ID'].isin(excluded_merchants)]


### Create a Total Amount column + Coefficient of Variation column

In [302]:
merged_df['Total Amount'] = merged_df.iloc[:, 7:].sum(axis=1)
merged_df['Coefficient of Variation'] = merged_df.iloc[:, 7:].std(axis=1) / merged_df.iloc[:, 7:].mean(axis=1)

### Merchant Grading

In [303]:
latest_transaction_month = trans_pivot_df.columns[-1]
def grade_merchant(row):
    if row[latest_transaction_month] > 5000000:
        return 'A'
    elif 500000 < row[latest_transaction_month] <= 5000000:
        return 'B'
    elif 100000 < row[latest_transaction_month] <= 500000:
        return 'C'
    elif 1000 < row[latest_transaction_month] <= 100000:
        return 'D'
    elif 0 < row[latest_transaction_month] <= 1000:
        return 'E'
    else:
        return 'F'
    
merged_df['Grade'] = merged_df.apply(grade_merchant, axis=1)

### Sort merchants by grade and within the same grade by 24 months total transaction amount

In [304]:
graded_merchants_df = merged_df.sort_values(by=['Grade', 'Total Amount'], ascending=[True, False])

### Mask the Shop Name

In [305]:
def mask_shop_name(name):
    if len(name) <= 5:
        return name[:-1] + 'X'
    return name[:5] + 'X' * (len(name) - 5)

graded_merchants_df['Shop Name'] = graded_merchants_df['Shop Name'].apply(mask_shop_name)

In [306]:
graded_merchants_df.to_csv('full_list.csv', index=False)

## filtered list

In [307]:
excluded_3000merchants = merchants_3000to_be_excluded['Merchant ID'].tolist()
final_merchants_df = graded_merchants_df[~graded_merchants_df['Merchant ID'].isin(excluded_3000merchants)]

In [308]:
active_merchants_df = final_merchants_df[final_merchants_df['status'] == 'ACTIVE']

In [309]:
filtered_df = active_merchants_df[(active_merchants_df['Age'] >= 24) & (active_merchants_df['Grade'].isin(['C', 'D', 'E', 'F']))]

In [310]:
filtered_df

,Create Date,Merchant ID,Shop Name,status,MCC,MCC Name,Age,2022-08,2022-09,2022-10,...,2024-01,2024-02,2024-03,2024-04,2024-05,2024-06,2024-07,Total Amount,Coefficient of Variation,Grade
10658,2022-08-18 10:39:57,10724,EASYHXXXXXXXXXXXXXXX,ACTIVE,5999,Miscellaneous and Specialty Retail Stores,24,27900.0,666644.0,786810.0,...,504830.0,370529.0,468850.0,206170.0,317150.0,439149.2,327605.0,13782007.00,2.406063,C
10563,2022-08-09 10:44:17,10629,CIU WXXXXXX,ACTIVE,5814,Fast Food Restaurants,24,22835.0,183293.0,397101.0,...,404194.0,401941.0,418937.0,363248.0,404946.0,401465.0,343419.0,9960587.14,2.400411,C
10666,2022-08-19 08:54:21,10732,YAU YXXXXXXXXX,ACTIVE,5814,Fast Food Restaurants,24,83584.0,361640.0,445012.4,...,363039.0,433795.0,357056.0,358369.0,416799.0,431769.0,389887.0,9570789.69,2.397993,C
10645,2022-08-16 18:18:32,10711,GENIUXXXXXXXXXXXXXXXX,ACTIVE,7298,Health and Beauty Spas,24,203158.0,320633.0,375117.0,...,367882.0,340785.0,393206.0,357548.0,292182.0,433933.0,244631.0,8144134.00,2.399013,C
10653,2022-08-17 16:15:48,10719,VICTOXXXXXXXXXXXXXXXXXXXXXXXXXXXXX,ACTIVE,5047,Dental/Laboratory/Medical/Ophthalmic Hospital ...,24,0.0,202985.0,222729.0,...,167542.0,179007.0,338801.0,242438.0,199039.0,192136.2,189254.0,6500202.59,2.404782,C
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19306,2022-04-27 15:40:35,1000045,KUDOSXXXXXXXXXXXXXXX,ACTIVE,5411,"Grocery Stores, Supermarkets",28,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,NaN,F
19308,2022-04-27 16:16:16,1000047,LIGHTXXXXXXXXXXXXXXXXXX,ACTIVE,5977,Cosmetic Stores,28,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,NaN,F
19313,2022-06-07 18:16:47,1000052,EXCELXXXXXXXXXXXXXXXXXX,ACTIVE,4812,Telecommunication Equipment Including Telephon...,26,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,NaN,F
19314,2022-06-09 15:33:59,1000053,SUNSTXXXXXXXXXXXXXXXXXX,ACTIVE,5039,Construction Materials Not Elsewhere Classified,26,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,NaN,F


In [311]:
filtered_df.to_csv('filtered_list.csv', index=False)